# Interfacing Python with compiled code, Cython

Interfacing python with compiled code lets you speed up some critical part of the code.
There are numerous ways to do this:

#### C API to Python and NumPy 

This is a library of C functions and variables that can be used
to create wrapper functions that together with the targeted C code can be compiled into fast
binary Python modules. See: https://docs.python.org/3/extending/extending.html for more information.

#### ctypes module and attribute 

The ctypes module from the Python standard library and the
ctypes attribute of NumPy arrays can be used to create a Python wrapper for an existing
dynamically-loaded library written in C.

#### Cython 

This facilitates the writing of C extensions for Python.
weave This allows the inclusion of C code in Python programs.

#### SWIG 

This automates the process of writing wrappers in C for C functions. SWIG is easy to
use if the argument list is to be limited to builtin Python types but can be cumbersome if
efficient conversion to NumPy arrays is desired. The difficulty is due to the need to match
C array parameters to predefined patterns in the numpy.

#### f2py 

This is for interfacing to Fortran.
See http://www.scipy.org/Topical_Software for links to some of these. Presented here is the
use of ctypes. Unlike the use of the C API or SWIG, it permits the interface to be written in
Python.



Let us start by writing some C code. The dot product of two vectors for instance:

```C
double dot_product(double v[], double u[], int n)
{
    double result = 0.0;
    for (int i = 0; i < n; i++)
        result += v[i]*u[i];
    return result;
}
```

Next we compile it, and build a shared object (please open another terminal window, not in the notebook):

```bash
gcc -c -Wall -Werror -fpic my_dot.c 
gcc -shared -o my_dot.so my_dot.o
```

The ctypes module of the Python standard library provides definitions of fundamental data types that can be passed to C programs. For example, assuming we


In [37]:
import ctypes as C
#these types would have names like C.c int and C.c double.
#They can be used constructors, e.g.,
x = C.c_double(2.71828)
#for which x.value returns the Python object.
print(type(2.71828))
print(type(x))

<class 'float'>
<class 'ctypes.c_double'>


In [39]:
#Fundamental types can be composed to get new types, e.g.,
xp = C.POINTER(C.c_double)(); 
xp.contents = x
print(xp)
print(x)

c_double(2.71828)


In [40]:
#or simply xp = C.POINTER(C.c_double)(x) . You can change the value of x using
xp[0] = 3.14159

In [44]:
#Array types can be created by \multiplying" a ctype by a positive integer, e.g.,
ylist = [1.,2.3,4.,5.]
n = len(ylist)
y = (C.c_double*n)()
y[:] = ylist
#or simply
y = (C.c_double*n)(*ylist)
print(ylist)
print(y[0])

[1.0, 2.3, 4.0, 5.0]
1.0


The asterisk is a Python operator for expanding the elements of a sequence into the arguments of a
function. Convert a C array back to a Python value or list by indexing it with an int or a slice.
The ctypes module has a utility subpackage to assist in locating a dynamically-loaded library,
e.g.,

In [48]:
import ctypes.util # an explicit import is necessary
C.util.find_library('my_dot')
#locates the C math library. 
#For loading a library there are constructors, e.g.,
myDL = C.CDLL('./my_dot.so')
print(myDL)
#which makes my a module-like object (a CDLL object to be precise).

<CDLL './my_dot.so', handle 10560dd60 at 0x105094fd0>


Similar to a Python module, myDL has as attributes function-like objects (C function pointers to
be precise) which have the same names as the C functions in the library, e.g., myDL.dot. These
function-like objects themselves have an attribute restype, which must be used to declare the type
of its result. For a C function whose result type is void, use None. 

Here is a full example:

In [8]:
import time
start = time.time()

from ctypes import CDLL, c_int, c_double
mydot = CDLL('my_dot.so').dot_product
def dot(vec1, vec2): # vec1, vec2 are lists
    n = len(vec1)
    mydot.restype = c_double
    return mydot((c_double*n)(*vec1), (c_double*n)(*vec2), c_int(n))

vec1 = [x for x in range(100000000)]
vec2 = [x for x in range(100000000)]
print(dot(vec1,vec2))
end = time.time()
print("Elaspsed time: ", end-start)

3.333333283334632e+23
Elaspsed time:  124.00988602638245


The arguments should be explicitly converted to the appropriate C type. 
The result is automatically converted to a regular Python type, based on the restype attribute.

**Warning.** If you use the extension .so for the name of a file, do not make its stem the same as a
.py file in the same directory, e.g., do not have both a funcs.py and a funcs.so. One convention
is to use funcs.py and funcs.so.


### Repeat the same in Cython

The fundamental nature of Cython can be summed up as follows: Cython is Python with C data types.
As Cython can accept almost any valid python source file, one of the hardest things in getting started is just figuring out how to compile your extension.

Here is the bare Python implementation of the dot product of two lists/vectors:

In [25]:
start = time.time()

#def frange(x, y, jump):
#    while x < y:
#        yield x
#    x += jump
       
def dot_product(vec1,vec2):
    result = 0.0
    n = len(vec1)
    for i in range(n):
        result += vec1[i]*vec2[i]
    return result

vec1 = [x for x in range(100000000)]
vec2 = [x for x in range(100000000)]
print(dot_product(vec1,vec2))
end = time.time()
print("Elaspsed time: ", end-start)

KeyboardInterrupt: 

Let us take the dot_product function and put it in the .pyx file:

```python
cimport cython


@cython.boundscheck(False) # Will not check indexing, so ensure indices are valid and non-negative
@cython.wraparound(False) # Will not allow negative indexing
@cython.cdivision(True) # Will not check for division by zero
def dot_product(vec1,vec2):
    cdef float result = 0.0
    cdef unsigned int n = len(vec1)

    for i in range(n):
        result += vec1[i]*vec2[i]

    return result
```

We would need a setup file in addition to that:

```python
from distutils.core import setup
from Cython.Build import cythonize

setup(
  name = 'my dot',
  ext_modules = cythonize("cython_dot.pyx")
)
```

save it in cython_setup.py file and build:

```bash
python cython_setup.py build_ext --inplace
```

you will now see the .so file appear in your folder.

In [18]:
start = time.time()
from cython_dot import dot_product
vec1 = [x for x in range(100000000)]
vec2 = [x for x in range(100000000)]
print(dot_product(vec1,vec2))
end = time.time()
print("Elaspsed time: ", end-start)

2.4287628162912635e+23
Elaspsed time:  51.588934898376465


How can we make Cython implementation even faster? use less python's generic data structures and more numpy arrays!
Change the cython_dot.pyx file to look like:

```python
cimport cython
import numpy as np
cimport numpy as np

DTYPE = np.float64
ctypedef np.float64_t DTYPE_t

@cython.boundscheck(False) # Will not check indexing, so ensure indices are valid and non-negative
@cython.wraparound(False) # Will not allow negative indexing
@cython.cdivision(True) # Will not check for division by zero
def dot_product(np.ndarray[DTYPE_t, ndim=1] vec1, np.ndarray[DTYPE_t, ndim=1] vec2):
    cdef float result = 0.0
    cdef unsigned int i
    cdef unsigned int n = vec1.shape[0]

    for i in range(n):
        result += vec1[i]*vec2[i]

    return result
```   

and change the cython_setup.py to looks like this:

```python
#!/usr/bin/env python3

from distutils.core import setup
from Cython.Build import cythonize
import numpy as np

setup(
  name = 'my dot',
  ext_modules = cythonize("cython_dot2.pyx"),
  include_dirs = [np.get_include()]
)
```

rebuild the shared object, and rerun:

In [26]:
start = time.time()
import numpy as np
from cython_dot2 import dot_product
vec1 = np.arange(100000000,dtype=float)
vec2 = np.arange(100000000,dtype=float)
print(dot_product(vec1,vec2))
end = time.time()
print("Elaspsed time: ", end-start)

2.4287628162912635e+23
Elaspsed time:  8.972913026809692


In [27]:
#no recompile with fast math and O3

start = time.time()
import numpy as np
from cython_dot2 import dot_product
vec1 = np.arange(100000000,dtype=float)
vec2 = np.arange(100000000,dtype=float)
print(dot_product(vec1,vec2))
end = time.time()
print("Elaspsed time: ", end-start)

2.4287628162912635e+23
Elaspsed time:  0.9875469207763672


Finally, try with OpenMP
https://clang-omp.github.io